# Creating Features

In [ ]:
import scripts.init_gpu as init_gpu
import scripts.init_dataset as init_dataset
from scripts.triplet_functions import n_neurons
import pandas as pd


init_gpu.initialize_gpus()

locations = ['LOC2', 'LOC3']

print("Loading Dataset...")
# load the dataset
df = pd.read_csv(
    f"../dataset/processed/{locations[0]}-{locations[1]}-scaled-balanced.csv")

length = len(df.columns) - 2  # subtract the two label columns

Num GPUs Available:  0
Loading Dataset...
Training Websites: [1309, 228, 51, 563, 501, 457, 285, 209, 1385, 1116, 178, 1209, 864, 65, 61, 191, 447, 476, 1034, 1232, 54, 1149, 407, 1466, 1330, 1436, 1490, 859, 451, 919, 1206, 569, 13, 326, 1429, 865, 696, 1468, 318, 440, 689, 1492, 189, 778, 198, 735, 704, 1236, 541, 88, 940, 1098, 255, 775, 161, 1130, 600, 1287, 1266, 740, 1182, 393, 142, 93, 1354, 466, 592, 163, 1482, 206, 1456, 1462, 928, 1301, 747, 333, 758, 727, 429, 1372, 546, 1399, 1327, 146, 1247, 1300, 350, 1093, 1495, 334, 946, 777, 552, 1310, 1140, 449, 1402, 664, 114, 469, 1486, 646, 821, 548, 135, 432, 1161, 644, 435, 1342, 1022, 810, 1316, 939, 292, 542, 1493, 505, 1478, 1103, 538, 1197, 877, 1195, 817, 741, 1404, 283, 1043, 1010, 186, 96, 224, 313, 1285, 327, 1487, 1221, 130, 788, 781, 1220, 958, 1083, 514, 1133, 23, 234, 1099, 1419, 1312, 1463, 1498, 601, 890, 323, 929, 6, 539, 1025, 365, 1039, 217, 1280, 611, 1308, 1338, 1415, 1477, 1366, 765, 330, 1104, 1086, 1, 1226, 

c:\Users\kaush\Documents\Python Projects\DoH-Synthesis\code\scripts\init_dataset.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.sort_values(by=["Location"], inplace=True)


,Location,Website,0,1,2,3,4,5,6,7,...,118,119,120,121,122,123,124,125,126,127
0,LOC2,0,-0.714089,0.541197,0.391921,-0.400778,-0.266345,-0.522526,0.023889,-0.261817,...,0.125986,0.10277,0.092495,0.043004,0.158904,0.119506,0.099445,0.083843,0.037226,0.152216
1,LOC2,1006,0.093792,-1.273349,-2.142801,1.205098,1.879002,2.139593,1.482513,2.382939,...,0.125986,0.10277,0.092495,0.043004,0.158904,0.119506,0.099445,0.083843,0.037226,0.152216
2,LOC2,1006,1.401091,-1.636258,0.527106,0.354928,-0.017609,-0.049260,-0.177300,-0.412692,...,0.125986,0.10277,0.092495,0.043004,0.158904,0.119506,0.099445,0.083843,0.037226,0.152216
3,LOC2,1006,-0.831599,0.662167,0.527106,0.354928,-0.017609,-0.049260,-0.177300,-0.412692,...,0.125986,0.10277,0.092495,0.043004,0.158904,0.119506,0.099445,0.083843,0.037226,0.152216
4,LOC2,1006,0.093792,-1.152379,-2.007616,0.827245,1.630266,1.666328,1.281324,2.311938,...,0.125986,0.10277,0.092495,0.043004,0.158904,0.119506,0.099445,0.083843,0.037226,0.152216


In [6]:
# dataset for the classification
source_location, target_location = locations

target_df = df[df['Location'] == target_location]
target_df.sort_values(by=['Website'], inplace=True)
target_df.reset_index(drop=True, inplace=True)
target_df.head(20)

source_df = df[df['Location'] == source_location]
source_df.sort_values(by=['Website'], inplace=True)
source_df.reset_index(drop=True, inplace=True)

source_df.info()

C:\Users\kaush\AppData\Local\Temp\ipykernel_42828\4085505363.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df.sort_values(by=['Website'], inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148500 entries, 0 to 148499
Columns: 130 entries, Location to 127
dtypes: float64(128), int64(1), object(1)
memory usage: 147.3+ MB


C:\Users\kaush\AppData\Local\Temp\ipykernel_42828\4085505363.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_df.sort_values(by=['Website'], inplace=True)


In [7]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

# Example function to extract features from a single trace.
def extract_features_from_trace(trace):
    """
    Given a trace (a list of integers representing TLS record sizes),
    extract features as a dictionary containing counts for:
      - Unigrams: individual TLS record sizes.
      - Bigrams: pairs of consecutive TLS record sizes.
      - Burst features: grouping consecutive packets with the same sign.
    """
    features = {}
    
    # 1. Unigrams: count each TLS record size.
    for token in trace:
        key = f"U_{token}"
        features[key] = features.get(key, 0) + 1

    # 2. Bigrams: count each pair of consecutive TLS record sizes.
    for i in range(len(trace) - 1):
        key = f"B_{trace[i]}_{trace[i+1]}"
        features[key] = features.get(key, 0) + 1

    # 3. Burst features:
    #    Define a burst as a sequence of consecutive packets with the same sign.
    bursts = []
    if trace:
        current_burst = [trace[0]]
        for num in trace[1:]:
            # Check if the current number has the same sign as the last number in current burst.
            if (num >= 0 and current_burst[-1] >= 0) or (num < 0 and current_burst[-1] < 0):
                current_burst.append(num)
            else:
                bursts.append(current_burst)
                current_burst = [num]
        bursts.append(current_burst)
    
    # Burst Unigrams: use the sum of values in each burst.
    burst_unigrams = [sum(burst) for burst in bursts]
    for token in burst_unigrams:
        key = f"BU_{token}"
        features[key] = features.get(key, 0) + 1

    # Burst Bigrams: count consecutive burst sums as pairs.
    for i in range(len(burst_unigrams) - 1):
        key = f"BB_{burst_unigrams[i]}_{burst_unigrams[i+1]}"
        features[key] = features.get(key, 0) + 1

    return features

# def extract_features_from_df(df):
#     # Helper function to extract the trace from a DataFrame row.
#     def get_trace_from_row(row):
#         """
#         Extracts the DNS trace from a DataFrame row.
#         Assumes that the trace values are stored in the columns starting at index 2.
#         Drops any NaN values and converts each entry to an integer.
#         """
#         trace = row.iloc[2:].dropna().tolist()
#         return [int(x) for x in trace]

#     # Iterate over the DataFrame rows.
#     feature_dicts = []
#     for idx, row in df.iterrows():
#         trace = get_trace_from_row(row)
#         feats = extract_features_from_trace(trace)
#         feature_dicts.append(feats)

#     # Use DictVectorizer to transform our list of feature dictionaries into a fixed-length matrix.
#     vectorizer = DictVectorizer(sparse=False)
#     X_features = vectorizer.fit_transform(feature_dicts)

#     # Now, X_features is a 2D numpy array where each row is a fixed-length vector representing one trace.
#     print("Feature matrix shape:", X_features.shape)
#     print("Feature names:", vectorizer.get_feature_names_out())
#     return X_features, vectorizer()



In [8]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

def extract_features_from_datasets(train_df, test_df):
    # Helper function to extract the trace from a DataFrame row.
    def get_trace_from_row(row):
        """
        Extracts the DNS trace from a DataFrame row.
        Assumes that the trace values are stored in the columns starting at index 2.
        Drops any NaN values and converts each entry to an integer.
        """
        trace = row.iloc[2:].dropna().tolist()
        return [int(x) for x in trace]
    
    # Process training dataset.
    train_feature_dicts = []
    for idx, row in train_df.iterrows():
        trace = get_trace_from_row(row)
        feats = extract_features_from_trace(trace)  # Assumes extract_features_from_trace is defined.
        train_feature_dicts.append(feats)
    
    # Process test dataset.
    test_feature_dicts = []
    for idx, row in test_df.iterrows():
        trace = get_trace_from_row(row)
        feats = extract_features_from_trace(trace)  # Assumes extract_features_from_trace is defined.
        test_feature_dicts.append(feats)
    
    # Use DictVectorizer: fit on training and transform both datasets.
    vectorizer = DictVectorizer(sparse=False)
    X_train = vectorizer.fit_transform(train_feature_dicts)
    X_test = vectorizer.transform(test_feature_dicts)
    
    print("Training feature matrix shape:", X_train.shape)
    print("Test feature matrix shape:", X_test.shape)
    print("Feature names:", vectorizer.get_feature_names_out())
    
    return X_train, X_test, vectorizer

X_train, X_test, vectorizer = extract_features_from_datasets(source_df, target_df)

Training feature matrix shape: (148500, 553)
Test feature matrix shape: (148500, 553)
Feature names: ['BB_-10_0' 'BB_-10_1' 'BB_-10_2' 'BB_-10_3' 'BB_-10_5' 'BB_-10_7'
 'BB_-10_8' 'BB_-11_0' 'BB_-11_1' 'BB_-11_2' 'BB_-11_3' 'BB_-11_5'
 'BB_-12_0' 'BB_-12_1' 'BB_-12_2' 'BB_-12_3' 'BB_-12_4' 'BB_-12_5'
 'BB_-12_6' 'BB_-12_7' 'BB_-13_0' 'BB_-13_1' 'BB_-13_2' 'BB_-13_3'
 'BB_-13_4' 'BB_-13_5' 'BB_-13_6' 'BB_-14_0' 'BB_-14_1' 'BB_-14_2'
 'BB_-14_3' 'BB_-15_0' 'BB_-15_1' 'BB_-15_2' 'BB_-15_3' 'BB_-15_4'
 'BB_-15_6' 'BB_-15_7' 'BB_-15_8' 'BB_-16_0' 'BB_-16_1' 'BB_-16_2'
 'BB_-16_3' 'BB_-17_0' 'BB_-17_1' 'BB_-17_2' 'BB_-18_0' 'BB_-18_1'
 'BB_-18_2' 'BB_-19_0' 'BB_-19_1' 'BB_-19_3' 'BB_-1_0' 'BB_-1_1'
 'BB_-1_10' 'BB_-1_12' 'BB_-1_16' 'BB_-1_2' 'BB_-1_25' 'BB_-1_3' 'BB_-1_4'
 'BB_-1_5' 'BB_-1_6' 'BB_-1_7' 'BB_-1_8' 'BB_-1_9' 'BB_-20_0' 'BB_-20_2'
 'BB_-21_0' 'BB_-21_1' 'BB_-22_0' 'BB_-23_0' 'BB_-24_0' 'BB_-25_0'
 'BB_-25_1' 'BB_-27_0' 'BB_-27_1' 'BB_-28_0' 'BB_-29_0' 'BB_-2_0'
 'BB_-2_1' 'BB_-2

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

y_train = source_df['Website'].values
y_test = target_df['Website'].values

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report for Random Forest:\n", classification_report(y_test, y_pred_rf))

MemoryError: could not allocate 1572864000 bytes